In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd

# WebDriver 설정
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

branches = {
    'Branch1': 'https://m.place.naver.com/restaurant/1193586093/review/visitor?entry=pll',
    'Branch2': 'https://m.place.naver.com/restaurant/1193586094/review/visitor?entry=pll',
}

all_reviews = []

for branch, url in branches.items():
    driver.get(url)
    time.sleep(5)

    # "더보기" 버튼을 클릭하는 작업을 반복 (최대 시도 횟수 설정)
    max_attempts = 100  # 최대 100번 시도 (필요에 따라 조정 가능)
    attempts = 0
    while attempts < max_attempts:
        try:
            # "더보기" 버튼이 나타날 때까지 기다림
            more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, 'fvwqf'))
            )
            more_button.click()
            time.sleep(2)  # 페이지가 로드될 시간을 줌
            attempts += 1
        except Exception as e:
            print(f"{branch} 더보기 버튼을 찾을 수 없습니다. 더 이상 리뷰가 없을 수 있습니다.")
            break

    # 페이지 소스를 가져와서 BeautifulSoup로 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    reviews = soup.find_all('li', {'class': 'owAeM'})  # 리뷰 항목의 클래스 이름

    for review in reviews:
        username = review.find('span', class_='P9EZi').text if review.find('span', class_='P9EZi') else 'No Username'
        content = review.find('span', class_='zPfVt').text if review.find('span', class_='zPfVt') else 'No Content'
        date = review.find('time').text if review.find('time') else 'No Date'
        all_reviews.append({'Branch': branch, 'Username': username, 'Date': date, 'Review': content})

driver.quit()

# 수집된 리뷰를 DataFrame으로 변환 후 CSV 파일로 저장
df = pd.DataFrame(all_reviews)
df.to_csv('all_reviews.csv', index=False)

print("리뷰 크롤링 완료. CSV 파일로 저장되었습니다.")


KeyboardInterrupt: 